In [2]:
import xml.etree.ElementTree as ET
import duckdb
import os
from duckdb import ConstraintException, FatalException
from pathlib import Path
from tqdm import tqdm

In [3]:
# Connect to sql database
con = duckdb.connect(database='german-parliament', read_only=False)

In [4]:
def create_db(reset_db=False):
    """
    Creates the necessary tables in the DuckDB database for storing plenary minutes.
    This function creates the 'party', 'speaker', and 'plenary_minute' tables, as well as sequences for party and speaker IDs.

    Args:
        reset_db (bool): If True, drops existing tables and sequences before creating new ones.
                          Defaults to False.

    Returns:
        None
    """
    if reset_db:
        con.execute("DROP TABLE IF EXISTS speech;")
        con.execute("DROP TABLE IF EXISTS speaker;")
        con.execute("DROP TABLE IF EXISTS party;")
        con.execute("DROP TABLE IF EXISTS plenary_minute;")
        con.execute("DROP TABLE IF EXISTS session;")


        con.execute("DROP SEQUENCE IF EXISTS party_id_seq;")
        con.execute("DROP SEQUENCE IF EXISTS speaker_id_seq;")
        con.execute("DROP SEQUENCE IF EXISTS legislative_period_id_seq;")
        con.execute("DROP SEQUENCE IF EXISTS session_id_seq;")
        con.execute("DROP SEQUENCE IF EXISTS speech_id_seq;")

    # Create a sequence for party IDs to ensure unique IDs for each party. It's like an auto-incrementing primary key.
    con.execute("CREATE SEQUENCE IF NOT EXISTS party_id_seq START 1;")
    # Create the party which will be used to identify a party
    con.execute(
        """
        CREATE TABLE IF NOT EXISTS party (
            id INTEGER DEFAULT nextval('party_id_seq') PRIMARY KEY,
            -- If we insert a party we just use DEFAULT to get the next value from the sequence.
            name VARCHAR UNIQUE NOT NULL
        );
        """
    )

    # Create a sequence for speaker IDs to ensure unique IDs for each speaker.
    con.execute("CREATE SEQUENCE IF NOT EXISTS speaker_id_seq START 1;")
    # Create the speaker which will be used to identify a speaker
    con.execute("""
        CREATE TABLE IF NOT EXISTS speaker (
            id INTEGER DEFAULT nextval('speaker_id_seq') UNIQUE NOT NULL,
            -- If we insert a speaker we just use DEFAULT to get the next value from the sequence.
            name VARCHAR NOT NULL,
            party_id INTEGER REFERENCES party(id) NOT NULL,
            PRIMARY KEY (name, party_id)
        );
        """
    )

    # Create a sequence for legislative periods to ensure unique IDs for each legislative period.
    con.execute("CREATE SEQUENCE IF NOT EXISTS legislative_period_id_seq START 1;")
    # Create the session table to store information about legislative periods and sessions.
    con.execute("""
        CREATE TABLE IF NOT EXISTS session (
            id INTEGER DEFAULT nextval('legislative_period_id_seq') UNIQUE NOT NULL,
            legislative_period INTEGER NOT NULL,
            session_no INTEGER NOT NULL,
            date DATE NOT NULL,
            primary key (legislative_period, session_no)
        );
        """
    )

    # Create a sequence for plenary minute IDs to ensure unique IDs for each plenary minute.
    con.execute("CREATE SEQUENCE IF NOT EXISTS speech_id_seq START 1;")
    con.execute("""
        CREATE TABLE IF NOT EXISTS speech (
            id BIGINT DEFAULT nextval('speech_id_seq') PRIMARY KEY,
            session_id INTEGER REFERENCES session(id) NOT NULL,
            speaker_id INTEGER REFERENCES speaker(id) NOT NULL,
            role VARCHAR, -- Role of the speaker, e.g. 'mp'
            position VARCHAR, -- Position of the speaker, e.g. 'Präsident'
            content TEXT NOT NULL
        );
        """
    )
    con.commit()

In [5]:
def parse_xml(file_path):
    """
    Parses an XML file and returns the root element.

    Args:
        file_path (str): The path to the XML file.

    Returns:
        xml.etree.ElementTree.Element: The root element of the parsed XML tree,
                                         or None if parsing fails.
    """
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()
        return root
    except ET.ParseError as e:
        print(f"Error parsing XML file: {e}")
        return None
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None

In [6]:
def insert_session(legislative_period, session, date):
    """ Inserts session information into the session table.

    Args:
        meta_data (xml.etree.ElementTree.Element): The title statement element containing legislative period and session information, as well as the date.

    Raises:
        Exception: If the titleStmt is None or if the publicationStmt is not found in the XML file.

    Returns:
        None
    """
    # print("------------------------------------")
    # print(f"Legislative Period: {legislative_period}, Session: {session}")
    # print(f"Publication Date: {date}")#
    # print("------------------------------------\n")

    # Insert session into the session table(

    sql = """
        INSERT INTO session (legislative_period, session_no, date)
        VALUES (?, ?,?)
        ON CONFLICT DO NOTHING;
    """

    params = (
        legislative_period,
        session,
        date
    )
    con.execute(sql, params)

    # con.execute(f"""
    #     INSERT INTO session (id, legislative_period, session_no, date)
    #     VALUES (DEFAULT, '{legislative_period}', '{session}', '{date}');
    # """)


def insert_party(attributes):
    # Insert party into the party table if it does not exist. The on conflict do nothing clause ensures that our key constraints aren't violated
    sql = "INSERT INTO party (name) VALUES (?) ON CONFLICT DO NOTHING;"
    con.execute(sql, (attributes["party"],))


def insert_speaker(attributes):
    # Insert speaker into the speaker table
    sql = """
        INSERT INTO speaker (name, party_id)
        VALUES (
            ?,                                    -- speaker’s full name
            (SELECT id FROM party WHERE name = ?) -- look-up party once, safely
        ON CONFLICT DO NOTHING
        );
        """

    con.execute(sql, (attributes["name"], attributes["party"]))




def insert_speech(speech, attributes, legislative_period, session):
    # Insert speech into the speech table
    # print(attributes)
    speech_string = ET.tostring(speech, encoding='unicode') if speech is not None else ''

    party_id = con.execute(f"""
                    SELECT id FROM party WHERE name = '{attributes['party']}';
                """).fetchone()[0]
    speaker_id = con.execute(f"""
                    SELECT id FROM speaker WHERE name = '{attributes['name']}' AND party_id = {party_id};
                """).fetchone()[0]
    session_id = con.execute(f"""
                    SELECT id FROM session WHERE legislative_period = '{legislative_period}' AND session_no = '{session}';
                 """).fetchone()[0]

    sql = """
    INSERT INTO speech
           (session_id, speaker_id, role, position, content)
    VALUES (?, ?, ?, ?, ?)
    ON CONFLICT DO NOTHING;
    """

    # values to bind — order must match the placeholders
    params = (
        session_id,
        speaker_id,
        attributes.get("role"),      # None → NULL in SQL
        attributes.get("position"),
        speech_string
    )

    con.execute(sql, params)




def parse_plenary_minute(root):
    """ Parses the plenary minute XML root element and extracts speeches, speakers, and parties and inserts them into the database.

    Args:
        root (xml.etree.ElementTree.Element): The root element of the XML tree containing the data of a single plenary minute.

    Raises:
        Exception: If the XML root element is None.

    Returns:
        None

    """
    if root is not None:
        # Get the meta data information, i.e. date, session, etc.
        meta_data = root.find('teiHeader').find('fileDesc')
        
        titleStmt = meta_data.find('titleStmt')
        publicationStmt = meta_data.find('publicationStmt')
        
        legislative_period = titleStmt.find('legislativePeriod').text
        session = titleStmt.find('sessionNo').text
        date = publicationStmt.find('date').text

        # Insert session entry into the db
        insert_session(legislative_period, session, date)
        #Get the actual content of the plenary minute
        plenary_minute = root.find('text').find('body').findall('div')
        speeches = []
        # Get all 'div' elements in the 'body' of the 'text'. Those are the elements that contain the actual plenary minute content.
        for div in plenary_minute:
            speeches.extend(div.findall('sp'))

        for speech in speeches:
            # Get attributes of the speech element
            attributes = speech.attrib
            # Insert party of speaker into db
            insert_party(attributes)
            # Insert speaker (politician) into db
            insert_speaker(attributes)
            # Insert the speech into the db
            insert_speech(speech, attributes, legislative_period, session)
    else:
        raise Exception("Missing XML root element.")

def parse_legislative_period(path):
    print(f"Parsing legislative period at path: {path}")
    # We materialise the generator once, to obtain all entries, so we can call len() on it
    files = list(Path(path).rglob("*"))
    # tqdm is used to create a nice loading bar for progress
    for file in tqdm(files, total=len(files), desc="Processing files"):
        if file.suffix == ".xml":
            # print(f"Parsing file: {file}")
            root = parse_xml(file)
            parse_plenary_minute(root)
            # print(f"Parsed file: {file}")

    print(f"Finished parsing legislative period at path: {path}")



def main():
    # Create database tables
    create_db(True)
    # Parse each period between period 1 and 18
    for period in range(1, 19): 
        # 02:d adjusts the number to be two digits, e.g. 01, 02, ..., 10, 11, ...
        try:
            con.execute("BEGIN TRANSACTION;")
            parse_legislative_period(f'../data/plenary_minutes/wahlperiode_{period:02d}')
            con.execute("COMMIT;")
        except FatalException as e:
            con.execute("ROLLBACK;")
            print(f"Error processing legislative period {period}: {e}")


if __name__ == "__main__":
    main()

# Always close connection to ensure that we won't have problems reconnecting to the db!
con.close()

Parsing legislative period at path: ../data/plenary_minutes/wahlperiode_01


Processing files:   0%|          | 0/282 [00:01<?, ?it/s]


TransactionException: TransactionContext Error: Current transaction is aborted (please ROLLBACK)

In [1]:
con.close()

NameError: name 'con' is not defined

In [ ]:
con.execute("select * from speaker join party on speaker.party_id = party.id ;").fetchdf()